<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/analyse_NY_mobility_poverty_time_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [3]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 163 µs (started: 2021-04-15 04:02:54 +00:00)


In [4]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 36.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1732bdf4175f446749e08cca72c158bf56b62605456babe615ae68c27bb165c2
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [5]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 142 ms (started: 2021-04-15 04:03:51 +00:00)


In [6]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 302 ms (started: 2021-04-15 04:03:51 +00:00)


In [7]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.85 s (started: 2021-04-15 04:03:51 +00:00)


In [8]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 2min 45s (started: 2021-04-15 04:03:56 +00:00)


In [9]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 4.61 ms (started: 2021-04-15 04:06:42 +00:00)


In [10]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [11]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [12]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 2.89 ms (started: 2021-04-15 04:07:11 +00:00)


In [13]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|                              8|
|360850170103|2020-02-03T00:00:...|         250|                          68|                             41|                             26|
|360050213021|2020-02-03T00:00:...|          92|                          26|                             11|                              6|
|36005

In [14]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 13.6 ms (started: 2021-04-15 04:07:12 +00:00)


In [15]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|       

In [16]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 4.71 ms (started: 2021-04-15 04:07:13 +00:00)


In [17]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 29.2 ms (started: 2021-04-15 04:07:13 +00:00)




---


demographic data






---



In [18]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [19]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|poverty_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125| 19.22705314009662|
|360810384001|2020-02-03T00:00:...|     

In [20]:

#grouped_df_mob_demo.show()

time: 1.04 ms (started: 2021-04-15 04:07:15 +00:00)


In [21]:
# Count number of rows
#spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

time: 1 ms (started: 2021-04-15 04:07:15 +00:00)


In [27]:
df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 30')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


#grouped_df_mob_demo_high_class.printSchema()


df_mob_demo_middle_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage >= 30 AND poverty_percentage < 70 ')
grouped_df_mob_demo_middle_class = df_mob_demo_middle_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_middle_class.createOrReplaceTempView('mob_demo_middle_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage >= 70')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

#df_mob_demo_merhed = spark.sql('SELECT h.*, m.')


grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as high_comp_home_perc,
m.completely_home_percentage as middle_comp_home_perc,
l.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_middle_class m ON h.date = m.date INNER JOIN mob_demo_low_class l ON m.date = l.date

""")



time: 586 ms (started: 2021-04-15 04:08:06 +00:00)


In [23]:
#Sort by poverty
# df_mob_demo = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
# df_mob_demo.createOrReplaceTempView('demographic_mobility')
# df_mob_demo.show()

time: 4.73 ms (started: 2021-04-15 04:07:15 +00:00)


In [29]:
grouped_df_mob_demo_combined.show()


+--------------------+-------------------+---------------------+------------------+
|                date|high_comp_home_perc|middle_comp_home_perc|low_comp_home_perc|
+--------------------+-------------------+---------------------+------------------+
|2020-02-26T00:00:...|  21.44108061674032|   25.201892355848827|22.889355039585393|
|2020-07-03T00:00:...| 37.182673621044614|   39.437899501922836| 35.58971321673658|
|2020-10-18T00:00:...|  37.91325824809544|    41.38024430431875| 36.92008749914052|
|2020-07-14T00:00:...|  37.06030552656261|   38.806733019017855|34.110554258248364|
|2020-03-01T00:00:...|  34.68441769282517|    39.01398352324866| 39.45405865775192|
|2020-04-09T00:00:...|  56.00597441488781|    57.68649004676348| 52.24652971742401|
|2020-10-09T00:00:...| 31.536565476466453|   34.296374770547665| 32.22105165455139|
|2020-11-22T00:00:...| 36.743866536295286|    39.83151681506591|37.653575463957594|
|2020-12-27T00:00:...|  37.17458663545729|    40.64745707020377| 34.59179776

In [30]:
count = grouped_df_mob_demo_combined.count()

time: 4min 48s (started: 2021-04-15 04:28:02 +00:00)


In [31]:
print("{}".format(count))

366
time: 1.72 ms (started: 2021-04-15 04:36:15 +00:00)


In [34]:
grouped_df_mob_demo_combined_pandas = grouped_df_mob_demo_combined.toPandas()
grouped_df_mob_demo_combined_pandas

,date,high_comp_home_perc,middle_comp_home_perc,low_comp_home_perc
0,2020-02-26T00:00:00-05:00,21.441081,25.201892,22.889355
1,2020-07-03T00:00:00-04:00,37.182674,39.437900,35.589713
2,2020-10-18T00:00:00-04:00,37.913258,41.380244,36.920087
3,2020-07-14T00:00:00-04:00,37.060306,38.806733,34.110554
4,2020-03-01T00:00:00-05:00,34.684418,39.013984,39.454059
...,...,...,...,...
361,2020-04-16T00:00:00-04:00,54.274207,55.907929,51.203304
362,2020-04-30T00:00:00-04:00,49.950118,51.735755,46.371618
363,2020-06-14T00:00:00-04:00,42.583273,45.026089,42.473243
364,2020-12-14T00:00:00-05:00,40.202016,41.640019,37.635765


time: 9.65 s (started: 2021-04-15 04:45:58 +00:00)


---
graphs 

---

In [39]:
base = alt.Chart(grouped_df_mob_demo_combined_pandas.reset_index()).encode(x='date')

alt.layer(
    base.mark_line(color='blue').encode(y='high_comp_home_perc'),
    base.mark_line(color='red').encode(y='middle_comp_home_perc'),
    base.mark_line(color='green').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 141 ms (started: 2021-04-15 04:57:36 +00:00)


In [ ]:
# alt.Chart(result_mean_df).mark_bar(size=30).encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_completely_home_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_completely_home_percentage')]
# ).properties(width=400, height=400).interactive()

In [ ]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_completely_home_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_completely_home_percentage')]
# ).properties(width=400, height=400).interactive()



In [ ]:
# result_df['poverty_percentage'].astype(float).dtype

In [ ]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_full_time_work_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_full_time_work_percentage')]
# ).properties(width=400, height=400).interactive()

In [ ]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_part_time_work_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_part_time_work_percentage')]
# ).properties(width=400, height=400).interactive()